# Information Aggregator

#### Author: Iain Muir, iam9ez@virginia.edu  
*Date: January 20th, 2021  
Project: Information Aggregator Application — powered by Datapane*

## Sources

* **[Robinhood](https://robin-stocks.readthedocs.io/en/latest/index.html)**
* **[Finnhub](https://finnhub.io/docs/api/)**
* **[ESPN](https://gist.github.com/akeaswaran/b48b02f1c94f873c6655e7129910fc3b)**
* **[The New York Times](https://developer.nytimes.com/apis)**
* **[CNN Money](https://money.cnn.com/data/markets)**
* **[FiveThirtyEight](https://data.fivethirtyeight.com/)**
* **[COVID Act Now](https://apidocs.covidactnow.org/api)**
* **[Lichess](https://lichess.org/api)**
* **[Spotify](https://developer.spotify.com/documentation/)**
* **[YouTube]()**

## Table of Contents <a class="anchor" id="toc"></a>

* **[0. Import Libraries and Secrets](#import)**  
* **[1. Build Report Components](#build)**  
    * [1.1 Header](#header)  
    * [1.2 Main Selector](#selector)  
        * *[1.2.1 Sports Results and Analytics](#espn)*  
        * *[1.2.2 Top World News](#nyt)*  
        * *[1.2.3 Stock Market](#finnhub)*  
        * *[1.2.4 Data Exploration](#data)*  
        * *[1.2.5 Compiled Selector](#compiled)*  
    * [1.3 Credits](#credits)  
* **[2. Datapane Report](#report)**  

## 0. Import Libraries and Secrets <a class="anchor" id="import"></a>

[Table of Contents](#toc)

Importing Standard Libraries...

In [1]:
from itertools import zip_longest
from bs4 import BeautifulSoup
from functools import partial
import datapane as dp
import pandas as pd
import numpy as np
import webbrowser
import warnings
import datetime
import requests
import time
import json

Importing Modules...

In [2]:
from robinhood import ticker_toggle, make_header, authenticate_, load_portfolio, get_scroll_objects, robinhood_news
from espn import format_news, news, parse_team, format_scores, scores, group_sport, standings
from finnhub import quote, candles, candlestick, name_search, big_number, profile, news
from nyt import top_stories, semantics, format_article, format_sections
from cnn_money import cnn_big_numbers, format_modules, group_modules
from fivethirtyeight import active_teams, make_button, plot_elo
from covid import county_choropleth, plot_state, altair_line
from spotify import get_spotify_embed

In [3]:
from errors import ErrorHandler, Logging, get_error_info 

In [4]:
from constants import ROOT, API_LOGOS, ESPN_SPORTS

Checking Datapane version...

In [5]:
version = !datapane --version
version_num = float(version[0].split()[2][2:])
version_num

14.0

In [6]:
assert version_num >= 13.0

In [7]:
!datapane login --token=55010cebc170ecfbeddb82838c360776bf36f6be

Connected successfully to https://datapane.com as iam9ez@virginia.edu


Open and Unpack Secrets File...

In [8]:
with open('secrets.json') as s:
    secrets = json.loads(s.read())

In [9]:
# Datapane
DATAPANE_KEY = secrets['datapane']
    
# Finnhub
FINNHUB_KEY = secrets['finnhub']

# Robinhood
ROBIN_USERNAME, ROBIN_PASSWORD = secrets['robinhood'].values()

# New York Times
NYT_KEY,  NYT_SECRET, NYT_ID = secrets['nyt'].values()

# Spotify
SPOTIFY_SECRET, SPOTIFY_ID = secrets['spotify'].values()

# COVID Act Now
COVID_KEY = secrets['covid']

In [10]:
AUTH_URL = 'https://accounts.spotify.com/api/token'
auth_response = requests.post(
    AUTH_URL, 
    {
        'grant_type': 'client_credentials',
        'client_id': SPOTIFY_ID,
        'client_secret': SPOTIFY_SECRET
    }
)
auth_response_data = auth_response.json()
access_token = auth_response_data['access_token']

In [11]:
try:
    robinhood = authenticate_(ROBIN_USERNAME, ROBIN_PASSWORD)
except KeyError:
    print(
        ErrorHandler("Failed Robinhood Authentication", *get_error_info())
    )
    robinhood = None

In [12]:
assert robinhood is not None

Preferences

In [13]:
warnings.simplefilter(action="ignore", category=pd.core.common.SettingWithCopyWarning)

## 1. Build Report Compenents <a class="anchor" id="build"></a>

[Table of Contents](#toc)

In [14]:
TODAY = datetime.date.today()

### 1.1 Header <a class="anchor" id="header"></a>

In [15]:
datapane, nyt, espn, spotify, _538, lichess = API_LOGOS.values()

In [16]:
header_logo = dp.HTML(
    """
    <html>
        <style type='text/css'>
            .images {
                display:flex;
                justify-content:center;
                align-items:center;
            }
            .images img {
                margin-left:5px;
                margin-right:5px;
            }
        </style>
        
        <center>
            <div class='images'>
                <img src='""" + datapane + """' width="75"/>
                <img src='""" + nyt + """' width="100"/>
                <img src='""" + espn + """' width="90"/>
                <img src='""" + spotify + """' width="90"/>
                <img src='""" + _538 + """' width="90"/>
            </div>
        </center>
    </html>
    """
)

In [17]:
header_text = dp.HTML(
    """
    <html>
        <style type='text/css'>
            @keyframes rotate {
                0%   {color: #EEE;}
                25%  {color: #EC4899;}
                50%  {color: #8B5CF6;}
                100% {color: #EF4444;}
            }
            h1 {
                color:#eee;
                animation-name: rotate;
                animation-duration: 4s;
                animation-iteration-count: infinite;
            }
        </style>
        <center>
            <h1>Morning Scoop</h1>
            <i>""" + TODAY.strftime('%A, %B %d, %Y') + """<i>
        </center>
    </html>
    """
)

In [18]:
header_description = dp.Text(
    """
Welcome to the Morning Scoop! This project is aimed to aggregate and display an eclectic and wide-ranging mix of information, data, and visualizations. Information sources include The New York Times, ESPN, FiveThirtyEight, and many more. Enjoy!

This report is powered by Datapane.

    """.strip()
)

### 1.2 Main Selector <a class="anchor" id="selector"></a>

[Table of Contents](#toc)

#### 1.2.1 Sports Results and Analytics <a class="anchor" id="espn"></a>

Scores/Analytics/YouTube Highlights

In [19]:
try:
    espn_results = list(map(
        scores, ESPN_SPORTS.items()
    ))
    espn_results = list(map(
        lambda result: [dp.Text('No games played yesterday...')] if result == [] else result, espn_results
    ))
except:
    print(
        ErrorHandler("Failed to get ESPN Results", *get_error_info())
    )
    exit(0)

espn.py (Line: ~400) || CustomError: No Stats/Leaders for KC @ DET; OK.
espn.py (Line: ~400) || CustomError: No Stats/Leaders for MIA @ WSH; OK.
espn.py (Line: ~400) || CustomError: No Stats/Leaders for MIL @ PIT; OK.
espn.py (Line: ~400) || CustomError: No Stats/Leaders for TB @ TOR; OK.
espn.py (Line: ~400) || CustomError: No Stats/Leaders for ATL @ CIN; OK.
espn.py (Line: ~400) || CustomError: No Stats/Leaders for NYY @ CLE; OK.
espn.py (Line: ~400) || CustomError: No Stats/Leaders for TEX @ NYM; OK.
espn.py (Line: ~400) || CustomError: No Stats/Leaders for LAA @ HOU; OK.
espn.py (Line: ~400) || CustomError: No Stats/Leaders for BAL @ MIN; OK.
espn.py (Line: ~400) || CustomError: No Stats/Leaders for BOS @ CHC; OK.
espn.py (Line: ~400) || CustomError: No Stats/Leaders for ARI @ COL; OK.
espn.py (Line: ~400) || CustomError: No Stats/Leaders for CHW @ SF; OK.
espn.py (Line: ~400) || CustomError: No Stats/Leaders for OAK @ SEA; OK.
espn.py (Line: ~400) || CustomError: No Stats/Leaders 

Upcoming Schedule

In [20]:
try:   
    espn_upcoming = list(map(
        lambda sport: scores(sport, upcoming=True), ESPN_SPORTS.items()
    ))
    espn_upcoming = list(map(
        lambda game: [dp.Text('No upcoming games today...')] if game == [] else game, espn_upcoming
    ))
except:
    print(
        ErrorHandler("Failed to get ESPN Schedule", *get_error_info())
    )
    exit(0)

Standings

In [21]:
# records = list(map(
#     standings, ESPN_SPORTS.items()
# ))

##### Compiled ESPN Group

Order Results and Schedules by Active Leagues

In [22]:
active = [
    1 if not isinstance(result[0], dp.client.api.report.blocks.Text) else 0
    for result in espn_results
]
active = pd.DataFrame({
    'results': espn_results,
    'league': ESPN_SPORTS.keys(),
    'active': active
}).sort_values(
    by='active',
    ascending=False
)
espn_results = active.results.tolist()
league_results = active.league.tolist()

In [23]:
active = [
    1 if not isinstance(upcoming[0], dp.client.api.report.blocks.Text) else 0
    for upcoming in espn_upcoming
]
active = pd.DataFrame({
    'upcoming': espn_upcoming,
    'league': ESPN_SPORTS.keys(),
    'active': active
}).sort_values(
    by='active',
    ascending=False
)
espn_upcoming = active.upcoming.tolist()
league_upcoming = active.league.tolist()

In [24]:
assert league_upcoming == league_results

In [25]:
espn_groups = list(map(
    group_sport, league_results, espn_results, espn_upcoming
))

#### 1.2.2 Top World News <a class="anchor" id="nyt"></a>

In [26]:
articles = top_stories(NYT_KEY)

In [27]:
articles = articles.loc[articles['item_type'] != 'Promo']
articles = articles.loc[articles['multimedia'].str.len() >= 1]
articles['multimedia'] = articles['multimedia'].str[0].str['url']
articles['subsection'] = articles['subsection'].replace('', 'miscellaneous')

In [28]:
grouped_articles = articles.groupby(
    'subsection',
    axis=0
)

In [29]:
ordered_groups = grouped_articles.size().sort_values(ascending=False)

In [30]:
article_sections = list(map(
    partial(format_sections, grouped_articles), ordered_groups.index
))

In [31]:
# TODO Article Links

#### 1.2.3 Stock Market <a class="anchor" id="finnhub"></a>

##### Ticker Scroll

In [32]:
t100 = robinhood.markets.get_top_100()
t100 = pd.DataFrame(t100)

In [33]:
scroll_objects = t100[['last_trade_price', 'previous_close', 'symbol']].apply(
    get_scroll_objects, axis=1
)

In [34]:
scroll = dp.HTML(
    """
    <html>
        <style type='text/css'>
            body {
                background: #FFFFFF;
            }
            
            .news-scroll a {
                font-size: 16px;
                text-decoration: none;
                color: #000000;
            }
            
            .price {
                font-size: 14px;
            }
            
            .up {
                font-size: 12px;
                color: #228B22;
            }
            
            .down {
                font-size: 12px;
                color: #D2042D;
            }
        </style>
    
        <div class="container mt-5">
            <div class="row">
                <div class="col-md-12">
                    <div class="d-flex justify-content-between align-items-center breaking-news bg-white">
                        <marquee class="news-scroll" behavior="scroll" direction="left" onmouseover="this.stop();" onmouseout="this.start();">""" + 
                            ' '.join(scroll_objects) + 
                        """</marquee>
                    </div>
                </div>
            </div>
        </div>
    
    </html>
    """.strip()
)

##### FAANG and ETF Prices and Candlestick

In [35]:
COLUMNS = 5
TICKERS = [
    'SPY', 'QQQ', 'XLF', 'IWM', 'BND',
    'FB', 'AAPL', 'AMZN', 'NFLX', 'GOOG',
]

In [36]:
big_numbers = []
figures = []

for i, t in enumerate(TICKERS):
    close, delta, delta_pct, high, low, open_, p_close, _ = quote(FINNHUB_KEY, t)
    df = candles(FINNHUB_KEY, t)
    
    bn = dp.BigNumber(
        heading=t,
        value=f"${round(close, 2)}",
        change=f"{round(delta_pct, 2)}%",
        is_upward_change=True if delta_pct > 0 else False
    )
    big_numbers.append(bn)
    
    figure = candlestick(df, t)
    figures.append(figure)

In [37]:
ticker_groups = list(zip_longest(*(iter(big_numbers),) * COLUMNS, fillvalue=''))
ticker_groups = [
    dp.Group(columns=COLUMNS, *g) for g in ticker_groups
]

##### Scrape CNN Market

In [38]:
cnn = 'https://money.cnn.com/data/markets'
with requests.get(cnn) as page:
    soup = BeautifulSoup(page.content, 'html.parser')

In [39]:
modules = soup.find_all('div', class_='module')
key_stats = soup.find('ul', class_='module-body wsod key-stats')

In [40]:
pairs = list(map(format_modules, modules))
pairs = list(filter(None, pairs))
pairs = list(sum(pairs, ()))

In [41]:
headers = pairs[::2]
modules = pairs[1::2]

In [42]:
module_groups = list(map(group_modules, headers, modules))

In [43]:
headers

[]

##### Free Stock Search

In [44]:
%%time

# sp500_figures = free_stock_search(FINNHUB_KEY)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 7.87 µs


#### 1.2.4 Data Exploration and Visualization <a class="anchor" id="data"></a>

##### COVID

US KPIs

In [45]:
usa_link = f'https://api.covidactnow.org/v2/country/US.json?apiKey={COVID_KEY}'
with requests.get(usa_link) as r:
    usa = r.json()

In [46]:
stats = usa['actuals']

In [47]:
c, n_c, d, n_d = stats['cases'], stats['newCases'], stats['deaths'], stats['newDeaths']
vax_rat = usa['metrics']['vaccinationsCompletedRatio']

In [48]:
covid_kpi = dp.Group(
    dp.BigNumber(
        heading='Total Cases',
        value="{:,}".format(c),
        change=f"{round(n_c / c * 100, 2)}%",
        is_upward_change=True if n_c / c - 1 > 0 else False
    ),
    dp.BigNumber(
        heading='Total Deaths',
        value="{:,}".format(d),
        change=f"{round(n_d / d * 100, 2)}%",
        is_upward_change=True if n_d / d - 1 > 0 else False
    ),
    dp.BigNumber(
        heading='Vaccine Completion',
        value=f"{round(vax_rat * 100, 2)}%"
    ),
    columns=3
)

County Choropleth

In [49]:
geo_link = 'https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json'
with requests.get(geo_link) as r:
    geo_json = r.json()

In [50]:
counties = f'https://api.covidactnow.org/v2/counties.json?apiKey={COVID_KEY}'

In [51]:
with requests.get(counties) as r:
    counties = pd.DataFrame(r.json())

In [52]:
counties['cases'] = counties.actuals.str['cases']
counties['case_ratio'] = counties.cases / counties.population
counties['deaths'] = counties.actuals.str['deaths']
counties['death_ratio'] = counties.deaths / counties.population
counties['vaccine_ratio'] = counties.metrics.str['vaccinationsCompletedRatio']

In [53]:
%%time

covid_fig = county_choropleth(counties, geo_json)

CPU times: user 1.82 s, sys: 258 ms, total: 2.08 s
Wall time: 2.88 s


Individual State Plots

In [54]:
states = f'https://api.covidactnow.org/v2/states.timeseries.json?apiKey={COVID_KEY}'

In [55]:
with requests.get(states) as r:
    states = pd.DataFrame(r.json())

In [56]:
%%time

covid_figures = list(map(
    plot_state, states.state, states.actualsTimeseries
))

CPU times: user 24.1 s, sys: 552 ms, total: 24.6 s
Wall time: 25.2 s


##### Five Thirty Eight

In [57]:
NBA_ELO = 'https://projects.fivethirtyeight.com/nba-model/nba_elo.csv'
NFL_ELO = 'https://projects.fivethirtyeight.com/nfl-api/nfl_elo.csv'
MLB_ELO = 'https://projects.fivethirtyeight.com/mlb-api/mlb_elo.csv'
ELO_LINKS = [
    NBA_ELO, NFL_ELO, MLB_ELO
]

In [58]:
YEAR = 2021
NCAA = f'https://projects.fivethirtyeight.com/march-madness-api/{YEAR}/fivethirtyeight_ncaa_forecasts.csv'
POLLS = 'https://github.com/fivethirtyeight/data/tree/master/polls'

In [59]:
%%time

_538_figures = list(map(
    plot_elo, ELO_LINKS
))

CPU times: user 14.3 s, sys: 898 ms, total: 15.2 s
Wall time: 20.3 s


In [60]:
# TODO Clean and Style Graphs
    # Restyle Color
# TODO Header Blocks --> Intro ELO and By Sport
# Incorporate March Madness and ~Polls~

##### Spotify

In [61]:
TOP50 = '37i9dQZEVXbLRQDuF5jeBp'
MIX1 = '37i9dQZF1E35bNzojVbMHG'
MIX2 = '37i9dQZF1E38sJU7OsGSQi'
MIX3 = '37i9dQZF1E37xnKwvD1GJE'

PLAYLISTS = [
    TOP50, MIX1, MIX2, MIX3
]

In [62]:
HEADERS = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

In [63]:
playlists_html = list(map(get_spotify_embed, PLAYLISTS))

In [64]:
# TODO Correct Embedding...

##### Lichess

In [65]:
# TODO Top Player Matches

##### Data Exploration Compilation

In [66]:
covid_group = dp.Toggle(
    covid_kpi,
    dp.Divider(),
    dp.Plot(
        covid_fig
    ),
    dp.Divider(),
    dp.Select(
        *covid_figures
    ),
    label = 'COVID'
)
_538_group = dp.Toggle(
    dp.Select(
        *_538_figures
    ),
    label = 'Five Thirty Eight'
)
spotify_group = dp.Toggle(
    dp.Group(
        *playlists_html,
        columns=4
    ),
    label = 'Spotify'
)
# lichess_group = dp.Toggle(
#     dp.Text("Lichess", label='A'),
#     label = 'Lichess'
# )

In [67]:
data_groups = [
    covid_group,
    _538_group,
    spotify_group
]

#### 1.2.5 Compiled Selector <a class="anchor" id="compiled"></a>

In [68]:
sports = dp.Group(
    dp.Select(
        *espn_groups,
        type=dp.SelectType.DROPDOWN
    ),
    label='Sports: Results and Analysis',
)
news = dp.Group(
    dp.Select(*article_sections),
    label='Top World News'
)
market = dp.Group(
    scroll,
    dp.Divider(),
    *ticker_groups,
    dp.Divider(),
    dp.Select(
        *figures
    ),
    dp.Divider(),
    *module_groups[:2],
    dp.Group(
        *module_groups[2:],
        columns=4
    ),
#     dp.Select(
#         *sp500_figures
#     ),
    label='Stock Market'
)
data = dp.Group(
    *data_groups,
    label='Data Exploration'
)

In [69]:
main_select = dp.Select(
    blocks=[
        sports, news, market, data

    ],
    type=dp.SelectType.TABS,
    label='selector'
)

### 1.3 Credits <a class="anchor" id="credits"></a>

[Table of Contents](#toc)

In [70]:
credits = dp.Text(
    "Report built by Iain Muir.",
    name='credits'
)

## 2. Datapane Report <a class="anchor" id="report"></a>

[Table of Contents](#toc)

In [71]:
BLOCKS = [
    header_logo,
    header_text,
    header_description,
    dp.Divider(),
    main_select,
    dp.Divider(),
    credits
]

In [72]:
report = dp.Report(
    blocks=BLOCKS
)

In [73]:
try:
    report.upload(
        name='Information Aggregator', 
        open=False
    )
except requests.exceptions.HTTPError:
    print(
        ErrorHandler("Report Upload Error; FATAL.", *get_error_info())
    )
    time.sleep(5)
    
    report.upload(
        name='Information Aggregator', 
        open=False
    )

Uploading report and associated data - *please wait...*

Uploading files


Report successfully uploaded. View and share your report <a href='https://datapane.com/reports/BAmGp47/information-aggregator/' target='_blank'>here</a>, or edit your report <a href='https://datapane.com/reports/BAmGp47/information-aggregator/edit/' target='_blank'>here</a>.

In [74]:
report.save(
    path=f'{ROOT}/Output/Information-Aggregator.html'
)

Report saved to .//Users/iainmuir/PycharmProjects/Desktop/PersonalProjects/Information-Aggregator/Output/Information-Aggregator.html

In [75]:
webbrowser.open(
    report.web_url
)

True

In [76]:
Logging.write_success_to_log()